## Here we reduce and concatenate the stats

In [1]:
import pandas as pd
import numpy as np
import os
import json
from tqdm import tqdm

def get_fixed_column_names():
    """
    Generate a fixed list of column names to ensure consistency across all files
    """
    columns = []
    
    # Models (excluding model_5)
    models = [f"model_{i}" for i in range(1, 7) if i != 5]
    
    # Metrics for each model
    metrics = ['gold_lp', 'rank', 'Entropy', 'gold_lp_cumsum', 'gold_prob', 'gold_lp_normalized']
    
    # Statistics to compute
    stats = ['median', 'max', 'min', 'std', 
             'moment1', 'moment2', 'moment3', 'moment4', 'moment5', 
             'q95', 'q90', 'q85', 'q80', 'q20', 'q15', 'q10', 'q05']
    
    # Generate columns for model metrics
    for model in models:
        for metric in metrics:
            for stat in stats:
                columns.append(f"{model}_{metric}_{stat}")
    
    # Generate columns for KL divergence terms
    model_pairs = [(i, j) for i in range(1, 7) for j in range(i+1, 7) 
                  if i != 5 and j != 5]
    
    for i, j in model_pairs:
        for stat in stats:
            columns.append(f"kl_{i}_vs_{j}_{stat}")
    
    return columns

def get_label_from_json(trial_name, question_number, label_folder_path):
    """
    Get label from corresponding JSON file if label_folder_path is provided
    """
    if not label_folder_path:  # If no path provided
        return None
        
    json_path = os.path.join(
        label_folder_path,
        f"{trial_name}.json"
    )
    
    try:
        with open(json_path, 'r') as f:
            data = json.load(f)
            label = data[f"Q{question_number}"]["eval_databases"]
            # Explicitly handle "N/A" to prevent pandas from converting it
            if label == "N/A":
                return "N/A"  # Forces pandas to keep it as string
            return label
    except Exception as e:
        print(f"Error reading JSON for {trial_name}: {e}")
        return None

def extract_trial_and_question(file_path):
    """
    Extract trial name and question number from file path
    Example: path/to/NCT00001959/logprob_matrix_1.csv -> ("NCT00001959", 1)
    """
    try:
        # Get filename and remove extension
        filename = os.path.basename(file_path)
        # Extract question number
        question_num = int(filename.split('_')[-1].split('.')[0])
        # Extract trial name from path
        trial_name = file_path.split('/')[-2]  # Adjust this based on your actual path structure
        return trial_name, question_num
    except Exception as e:
        print(f"Error extracting trial and question from {file_path}: {e}")
        return None, None

def compute_statistics(series):
    """
    Compute various statistical measures for a series
    """
    mean = series.mean()
    
    stats = {
        'median': series.median(),
        'max': series.max(),
        'min': series.min(),
        'std': series.std(),
        
        # Central moments
        'moment1': mean,
        'moment2': ((series - mean) ** 2).mean(),
        'moment3': ((series - mean) ** 3).mean(),
        'moment4': ((series - mean) ** 4).mean(),
        'moment5': ((series - mean) ** 5).mean(),
        
        # Existing quantiles
        'q95': series.quantile(0.95),
        'q90': series.quantile(0.90),
        'q85': series.quantile(0.85),
        'q80': series.quantile(0.80),
        'q20': series.quantile(0.20),
        'q15': series.quantile(0.15),
        'q10': series.quantile(0.10),
        'q05': series.quantile(0.05),
    }    
    return stats

def process_metrics_and_kl(input_folder, kl_folder, output_folder, label_folder_path=None):
    """
    Process original metrics and KL divergence terms with consistent column ordering
    """
    # Get fixed column names
    fixed_columns = get_fixed_column_names()
    
    # Original metrics to analyze
    metrics = ['gold_lp', 'rank', 'Entropy', 'gold_lp_cumsum', 'gold_prob', 'gold_lp_normalized']
    
    # Get all CSV files from model1 directory
    csv_files = []
    model1_path = os.path.join(input_folder, "model_1")
    for root, dirs, files in os.walk(model1_path):
        # Skip hidden directories
        dirs[:] = [d for d in dirs if not d.startswith('.')]
        
        for file in files:
            if not file.startswith('.') and file.endswith('.csv'):
                csv_files.append((os.path.join(root, file), file))
    
    print(f"Found {len(csv_files)} CSV files to process")
    print(f"Output will have {len(fixed_columns) + 1} columns")  # +1 for label
    
    for file_path1, file_name in tqdm(csv_files, desc="Processing files"):
        try:
            # Get trial name and question number
            trial_name, question_num = extract_trial_and_question(file_path1)
            
            if trial_name and question_num and label_folder_path:
                # Get label from JSON only if path is provided
                label = get_label_from_json(trial_name, question_num, label_folder_path)
            else:
                label = None
            
            # Dictionary to store all metrics
            all_stats = {col: np.nan for col in fixed_columns}  # Initialize with NaN
            
            # Add label column
            all_stats['label'] = label
            
            # Process each model (excluding model_5)
            models = [f"model_{i}" for i in range(1, 7) if i != 5]
            
            # Step 1: Process original metrics for each model
            for model in models:
                file_path = os.path.join(input_folder, model, os.path.relpath(file_path1, model1_path))
                if not os.path.exists(file_path):
                    print(f"Skipping {file_name} - no matching file in {model}")
                    continue
                
                df = pd.read_csv(file_path)
                
                # Compute statistics for each metric
                for metric in metrics:
                    if metric in df.columns:
                        stats = compute_statistics(df[metric])
                        for stat_name, value in stats.items():
                            col_name = f"{model}_{metric}_{stat_name}"
                            all_stats[col_name] = value
            
            # Step 2: Process KL divergence terms
            kl_file_path = os.path.join(kl_folder, os.path.relpath(file_path1, model1_path))
            if os.path.exists(kl_file_path):
                kl_df = pd.read_csv(kl_file_path)
                
                # Get all KL columns
                kl_cols = [col for col in kl_df.columns if col.startswith('kl_')]
                
                # Compute statistics for each KL term
                for kl_col in kl_cols:
                    stats = compute_statistics(kl_df[kl_col])
                    for stat_name, value in stats.items():
                        col_name = f"{kl_col}_{stat_name}"
                        all_stats[col_name] = value
            
            # Create output DataFrame with fixed column order plus label
            columns_with_label = fixed_columns + ['label']
            result_df = pd.DataFrame([all_stats])[columns_with_label]
            
            # Create output directory structure
            output_file_path = os.path.join(output_folder, os.path.relpath(file_path1, model1_path))
            os.makedirs(os.path.dirname(output_file_path), exist_ok=True)
            
            # Save results
            result_df.to_csv(output_file_path, index=False)
            
        except Exception as e:
            print(f"\nError processing {file_name}:")
            print(f"Error type: {type(e)}")
            print(f"Error message: {str(e)}")
            continue

def concatenate_output_files(output_folder):
    """
    Concatenate all CSV files in Output_folder and its subfolders into a single DataFrame
    """
    # Get list of all CSV files
    csv_files = []
    for root, dirs, files in os.walk(output_folder):
        # Skip hidden directories
        dirs[:] = [d for d in dirs if not d.startswith('.')]
        
        for file in files:
            if not file.startswith('.') and file.endswith('.csv'):
                csv_files.append((os.path.join(root, file), file))
    
    print(f"Found {len(csv_files)} CSV files to concatenate")
    # Sort files alphabetically
    csv_files = sorted(csv_files)
    
    # Read and concatenate all files
    all_dfs = []
    for file_path, file_name in tqdm(csv_files, desc="Reading files"):
        try:
            df = pd.read_csv(file_path)
            
            # Add filename as a column (optional but useful for tracking)
            df['source_file'] = file_name
            
            # Add full path as a column (optional)
            df['file_path'] = os.path.relpath(file_path, output_folder)
            
            all_dfs.append(df)
            
        except Exception as e:
            print(f"\nError reading {file_name}:")
            print(f"Error type: {type(e)}")
            print(f"Error message: {str(e)}")
            continue
    
    # Concatenate all DataFrames
    if all_dfs:
        final_df = pd.concat(all_dfs, ignore_index=True)
        
        # Print some information about the final DataFrame
        print("\nFinal DataFrame info:")
        print(f"Shape: {final_df.shape}")
        print(f"Number of samples: {len(final_df)}")
        print(f"Number of features: {len(final_df.columns)}")
        
        if 'label' in final_df.columns:
            print("\nLabel distribution:")
            print(final_df['label'].value_counts())
        
        return final_df
    else:
        print("No files were successfully read!")
        return None

if __name__ == "__main__":
    # Paths
    input_folder = "Database_free_evaluation/Clinical_trials/Features/FEATURES_Gpt5/Paragraph_title_features"
    kl_folder = "kl_analysis_Paragraph_title_features"
    output_folder = "Output_folder_Paragraph_title"

     # Label folder path (set to None if not using labels)
    label_folder_path = "Database_dependent_evaluation/Clinical_trials/4-Evaluation/Evaluation_factual/Gpt5/Paragraph_level/Evaluation_title_Paragraph_GPT"
    # Or for other folders:
    # label_folder_path = None
    
    # Create output directory if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Step 1: Process files and create individual outputs
    process_metrics_and_kl(input_folder, kl_folder, output_folder, label_folder_path)

    # Step 2: Concatenate all output files
    final_df = concatenate_output_files(output_folder)
    
    # Optional: Save concatenated DataFrame
    if final_df is not None:
        final_df.to_csv("concatenated_results_Paragraph_title_Gpt5.csv", index=False)

Found 1500 CSV files to process
Output will have 681 columns


Processing files: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1500/1500 [02:44<00:00,  9.10it/s]


Found 1500 CSV files to concatenate


Reading files: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1500/1500 [00:10<00:00, 140.84it/s]



Final DataFrame info:
Shape: (1500, 683)
Number of samples: 1500
Number of features: 683

Label distribution:
label
hallucination    957
fact             540
Name: count, dtype: int64


In [2]:
final_df

,model_1_gold_lp_median,model_1_gold_lp_max,model_1_gold_lp_min,model_1_gold_lp_std,model_1_gold_lp_moment1,model_1_gold_lp_moment2,model_1_gold_lp_moment3,model_1_gold_lp_moment4,model_1_gold_lp_moment5,model_1_gold_lp_q95,...,kl_4_vs_6_q90,kl_4_vs_6_q85,kl_4_vs_6_q80,kl_4_vs_6_q20,kl_4_vs_6_q15,kl_4_vs_6_q10,kl_4_vs_6_q05,label,source_file,file_path
0,-0.002555,0.000000e+00,-19.942202,3.237032,-1.324681,10.328688,-117.663115,1.916203e+03,-3.349904e+04,0.000000e+00,...,0.322478,0.273671,0.198252,0.000002,2.673825e-07,1.383371e-07,5.730916e-08,fact,logprob_matrix_1.csv,NCT00001959/logprob_matrix_1.csv
1,-0.443523,0.000000e+00,-100.000000,17.210684,-5.278262,293.963631,-25714.653499,2.440374e+06,-2.310694e+08,0.000000e+00,...,0.371619,0.290020,0.237494,0.000091,3.067300e-05,7.372770e-06,5.163657e-07,hallucination,logprob_matrix_10.csv,NCT00001959/logprob_matrix_10.csv
2,-0.277729,0.000000e+00,-100.000000,19.870327,-5.677067,392.479726,-34858.320077,3.298713e+06,-3.110793e+08,0.000000e+00,...,0.413269,0.296209,0.254602,0.002974,2.967417e-04,9.562714e-06,8.209919e-07,hallucination,logprob_matrix_11.csv,NCT00001959/logprob_matrix_11.csv
3,-0.852039,-1.192093e-07,-100.000000,26.665244,-9.935672,699.925306,-56484.975866,5.146168e+06,-4.629185e+08,-1.646198e-05,...,0.546066,0.507862,0.481236,0.018862,9.324209e-03,7.637982e-04,6.056297e-05,hallucination,logprob_matrix_12.csv,NCT00001959/logprob_matrix_12.csv
4,-0.072655,0.000000e+00,-100.000000,20.028006,-6.069710,396.942696,-34430.698761,3.244876e+06,-3.046625e+08,0.000000e+00,...,0.394189,0.335043,0.304713,0.000024,5.952057e-06,2.207163e-06,5.268360e-08,hallucination,logprob_matrix_13.csv,NCT00001959/logprob_matrix_13.csv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,-0.002204,0.000000e+00,-100.000000,10.742269,-2.246543,114.762294,-10351.731022,1.005393e+06,-9.813261e+07,0.000000e+00,...,0.406825,0.238360,0.131957,0.000002,3.015074e-07,4.444115e-08,1.112983e-08,fact,logprob_matrix_5.csv,NCT04739800/logprob_matrix_5.csv
1496,-0.155733,0.000000e+00,-100.000000,11.875579,-2.726345,140.864246,-12948.411565,1.258375e+06,-1.223799e+08,0.000000e+00,...,0.338628,0.266274,0.193969,0.000158,4.219662e-05,4.860021e-06,3.436082e-07,fact,logprob_matrix_6.csv,NCT04739800/logprob_matrix_6.csv
1497,-0.021725,0.000000e+00,-14.123719,2.870307,-1.346026,7.996347,-67.600266,8.178574e+02,-1.015044e+04,0.000000e+00,...,0.378751,0.168140,0.139849,0.000324,4.234440e-05,2.071257e-05,3.723692e-06,hallucination,logprob_matrix_7.csv,NCT04739800/logprob_matrix_7.csv
1498,-0.348750,0.000000e+00,-100.000000,16.216446,-4.577614,261.219971,-23167.683992,2.211847e+06,-2.109779e+08,-5.364418e-08,...,0.377780,0.235627,0.219161,0.000870,1.695121e-04,1.990751e-05,9.854478e-07,hallucination,logprob_matrix_8.csv,NCT04739800/logprob_matrix_8.csv


In [7]:
import numpy as np
row = [-6.88, -100, -11.85, -100, -100]
mean= np.mean(row)
mean

-63.746

In [8]:
std = np.std(row)
std

44.4297069988088

In [9]:
(-6.88 - mean) / std

1.2799094083948073

In [6]:
import numpy as np

row = [-6.88, -100, -11.85, -100, -100]  # Fixed the typo in -11,85

# Calculate mean and std of the vector
mean = np.mean(row)
std = np.std(row)

print(f"Mean: {mean}")
print(f"Std: {std}")

# Normalize the first component
normalized_value = (row[0] - mean) / std

print(f"\nOriginal value: {row[0]}")
print(f"Normalized value: {normalized_value}")

Mean: -63.746
Std: 44.4297069988088

Original value: -6.88
Normalized value: 1.2799094083948073
